## Training on whole image folder

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
#import gdal
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, Reshape
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
#import tifffile

2023-05-24 15:01:26.953581: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 15:01:27.038811: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-24 15:01:27.038824: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-24 15:01:27.056546: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-24 15:01:27.540972: W tensorflow/stream_executor/platform/de

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the paths to your image data directories
x_train_dir = 'data/train_patch/'
y_train_dir = 'data/train_label_patch/'

# Set the image and batch size
image_size = (512, 512)  # Target image size for resizing
batch_size = 64

# Data augmentation and normalization configuration for x_train
x_train_generator = ImageDataGenerator(
    rescale=1.0 / 255,  # Scale pixel values to [0, 1]
    rotation_range=20,  # Random rotation between -20 and 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20% of the width
    height_shift_range=0.2,  # Randomly shift images vertically by 20% of the height
    horizontal_flip=True  # Randomly flip images horizontally
)

# Create an image generator for x_train
x_train_image_generator = x_train_generator.flow_from_directory(
    x_train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,  # Set to None to return only input images
    shuffle=False  # Set to False to ensure x_train and y_train match
)

# Data normalization configuration for y_train
y_train_generator = ImageDataGenerator(
    rescale=1.0 / 255  # Scale pixel values to [0, 1]
)

# Create an image generator for y_train
y_train_image_generator = y_train_generator.flow_from_directory(
    y_train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,  # Set to None to return only input images
    shuffle=False  # Set to False to ensure x_train and y_train match
)

# Ensure x_train and y_train generators have the same number of samples
assert len(x_train_image_generator) == len(y_train_image_generator), "Mismatch in x_train and y_train data"
train_data_generator = zip(x_train_image_generator, y_train_image_generator)

Found 58320 images belonging to 5 classes.
Found 58320 images belonging to 5 classes.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from tensorflow.keras import layers, models

def unet_model():
    model = Sequential()

    # Encoder
    #model.add(layers.Reshape((5000*5000*3, ), input_shape=(5000,5000, 3)))
    #model.add(Reshape((5000*5000*3,), input_shape=(5000,5000,3))) 

    model.add(Conv2D(64, 3, activation='relu', padding='same', input_shape = ((512, 512, 3))))
    model.add(Conv2D(64, 3, activation='relu', padding='same'))
    #model.add(Conv2D(64, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, 3, activation='relu', padding='same'))
    #model.add(Conv2D(128, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, 3, activation='relu', padding='same'))
    #model.add(Conv2D(256, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Decoder
    model.add(UpSampling2D(size=(2, 2)))
    #model.add(Conv2D(128, 3, activation='relu', padding='same'))
    model.add(Conv2D(128, 3, activation='relu', padding='same'))

    model.add(UpSampling2D(size=(2, 2)))
    #model.add(Conv2D(64, 3, activation='relu', padding='same'))
    model.add(Conv2D(64, 3, activation='relu', padding='same'))

    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(1, 3, activation='sigmoid', padding='same'))

    return model

In [4]:
model = unet_model()

2023-05-24 15:01:42.210643: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-24 15:01:42.210779: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-24 15:01:42.210888: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-05-24 15:01:42.210941: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-05-24 15:01:42.210988: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 512, 512, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 512, 512, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 256, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 256, 256, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 128, 128, 128)    0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 128, 256)     2

In [6]:
def loss(y_true, y_pred):
    def dice_loss(y_true, y_pred):
        y_pred = tf.math.sigmoid(y_pred)
        numerator = 2 * tf.reduce_sum(y_true * y_pred)
        denominator = tf.reduce_sum(y_true + y_pred)
        return 1 - numerator / denominator
    y_true = tf.cast(y_true, tf.float32)
    cross_entropy_loss = tf.nn.sigmoid_cross_entropy_with_logits(y_true, y_pred)
    total_loss = cross_entropy_loss + dice_loss(y_true, y_pred)
    return tf.reduce_mean(total_loss)

In [7]:
def iou_metric(y_true, y_pred):
    y_pred = tf.math.round(y_pred)
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred) - intersection
    iou = intersection / (union + K.epsilon())
    return iou

In [8]:
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=loss, metrics=['accuracy', iou_metric])

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Activation, BatchNormalization, Concatenate
from tensorflow.keras.applications import VGG19
from tensorflow.keras import backend as K

In [ ]:
# Use the generator to train the model
model.fit(
    train_data_generator,
    steps_per_epoch=len(x_train_image_generator),
    epochs=1, batch_size = 32, verbose=1,
)